In [1]:
# Import our dependencies
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# File to Load (Remember to Change These)
train_data = "cleaned_train.csv"

In [2]:
df_train_clean = pd.read_csv(train_data)

In [3]:
df_train_clean[0:5]

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,...,country_of_residence_United States,country_of_residence_Viet Nam,used_app_before_no,used_app_before_yes,test_taker_?,test_taker_Health care professional,test_taker_Others,test_taker_Parent,test_taker_Relative,test_taker_Self
0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
# Split our preprocessed data into our features and target arrays
y = df_train_clean.Autism_Diagnosis.values
X = df_train_clean.drop(columns="Autism_Diagnosis").values


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [5]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train_scaled.shape[1]


nn_1 = tf.keras.models.Sequential()

# First hidden layer
nn_1.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim= input_features))

# Second hidden layer
nn_1.add(tf.keras.layers.Dense(units=50, activation="sigmoid"))

# Third hidden layer
nn_1.add(tf.keras.layers.Dense(units=20, activation="relu"))

# Output layer
nn_1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_1.summary()

nn_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

fit_model = nn_1.fit(X_train_scaled, y_train, epochs=150)

model_loss, model_accuracy = nn_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               6600      
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 20)                1020      
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
Total params: 12,691
Trainable params: 12,691
Non-trainable params: 0
_________________________________________________________________
Epoch 1/150
19/19 [==============================] - 1s 2ms/step - loss: 0.6256 - accuracy: 0.6383
Epoch 2/150
19/19 [==============================] - 0s 2ms/step - loss: 0.4309 - acc

In [7]:
y_predict = nn_1.predict(X_test_scaled)

import numpy as np
Y_predict = np.round(y_predict)

from sklearn.metrics import classification_report
# Create and save the training classification report
training_report = classification_report(y_test, Y_predict)

# Print the training classification report
print(training_report)

7/7 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

         0.0       0.89      0.91      0.90       151
         1.0       0.71      0.65      0.68        49

    accuracy                           0.85       200
   macro avg       0.80      0.78      0.79       200
weighted avg       0.85      0.85      0.85       200



In [13]:
# Export our model to HDF5 file
nn_1.save("Neural_Network_Optimized_Resampled.h5")